## Environment Setup and Installation  

This notebook requires specific dependencies, which are listed in `requirements.txt`. 

In [ ]:
# Install required packages
# !pip install -r requirements.txt

## API Key Setup & Loading Environment Variables  

This notebook requires API keys for **LLM-based neuroscience agents**.  
We will:  
1. Load API keys from a `.env` file (if available).  
2. Check if required API keys (**OpenAI, Anthropic**) are set in the environment.  
3. Mask the keys for security before displaying their status.  

If an API key is missing, ensure it is stored in your **`.env` file** or manually set as an environment variable.

In [ ]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv
import anthropic
from crewai import Agent, Task, Crew
from langchain.tools import Tool
import matplotlib.pyplot as plt

# Load environment variables from .env file
load_dotenv()

# Access API key
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not ANTHROPIC_API_KEY:
    print("No Anthropic key found...")
else:
    print(ANTHROPIC_API_KEY[:4],"*********************")

if not OPENAI_API_KEY:
    print("No OpenAI key found...")
else:
    print(OPENAI_API_KEY[:4],"*********************")


# Set environment variable for crewai
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
# Uncomment and run to remove API keys from environment
# del os.environ["ANTHROPIC_API_KEY"]
# del os.environ["OPENAI_API_KEY"]

## Sample Data Generator

Creating simple sample data for our agents to work with.

In [ ]:
def generate_sample_brain_scan_data():
    """Generate sample brain scan data"""
    brain_scan = {
        "study_info": {
            "type": "fMRI",
            "date": "2023-09-15",
            "subject_id": "SUB123",
            "task": "Working memory task"
        },
        "findings": {
            "prefrontal_cortex": "Increased BOLD signal during task engagement",
            "hippocampus": "Moderate activation in bilateral hippocampi",
            "parietal_lobe": "Strong task-related activation in posterior parietal regions",
            "visual_cortex": "Expected activation pattern during stimulus presentation"
        },
        "behavioral_metrics": {
            "accuracy": 87.5,
            "reaction_time_ms": 428,
            "missed_trials": 3
        }
    }
    return brain_scan

def generate_sample_patient_symptoms():
    """Generate sample neurological symptom data"""
    symptoms = {
        "patient_info": {
            "age": 42,
            "sex": "Male",
            "history": "No significant neurological history"
        },
        "reported_symptoms": [
            "Intermittent headaches (3 months, gradually increasing)",
            "Sensitivity to bright light",
            "Occasional numbness in right hand",
            "Recent episodes of short-term memory issues"
        ],
        "neurological_exam": {
            "cranial_nerves": "Intact",
            "motor": "5/5 strength in all extremities",
            "sensory": "Slight decreased sensation in right fingertips",
            "reflexes": "2+ and symmetric",
            "coordination": "Normal finger-to-nose testing"
        }
    }
    return symptoms

# Generate sample data
brain_scan_data = generate_sample_brain_scan_data()
patient_symptoms = generate_sample_patient_symptoms()

# Display sample data
print("Sample data generated.")

## AI-Powered Neuroscience Analysis Tools  

To process our generated neuroscience data, we define two **AI-powered tools**:  

1. **Brain Imaging Analyzer** → Uses LLMs to interpret fMRI findings.  
2. **Neurological Symptom Interpreter** → Uses LLMs to analyze clinical symptoms and suggest possible conditions.  

These tools simulate how AI can assist researchers and clinicians in understanding **brain activity and neurological disorders**.

In [ ]:
# Initialize Anthropic client
client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

def analyze_imaging(scan_data_json):
    """Tool to analyze brain imaging data"""
    try:
        # Convert JSON string to dict if needed
        if isinstance(scan_data_json, str):
            scan_data = json.loads(scan_data_json)
        else:
            scan_data = scan_data_json
            
        # Prepare prompt for analysis
        prompt = f"""
        You are a neuroscientist specializing in functional brain imaging. Please analyze the following fMRI data:
        
        {json.dumps(scan_data, indent=2)}
        
        Provide a detailed analysis including:
        1. Key findings in each brain region
        2. Correlation between neural activity and behavioral performance
        3. Interpretation of the activation patterns in the context of working memory
        4. Suggestions for follow-up analyses
        
        Format your response as a structured report a researcher could use.
        """
        
        # Get analysis
        response = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=800,
            temperature=0,
            system="You are an expert neuroscientist providing analysis of brain imaging data.",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        
        return response.content[0].text
    
    except Exception as e:
        return f"Error analyzing brain data: {str(e)}"

def interpret_neurological_symptoms(symptoms_json):
    """Tool to interpret neurological symptoms"""
    try:
        # Convert JSON string to dict if needed
        if isinstance(symptoms_json, str):
            symptoms = json.loads(symptoms_json)
        else:
            symptoms = symptoms_json
            
        # Prepare prompt for symptom interpretation
        prompt = f"""
        You are a neurologist specializing in clinical diagnosis. Please analyze these neurological symptoms:
        
        {json.dumps(symptoms, indent=2)}
        
        Provide:
        1. Interpretation of the presenting symptoms
        2. Potential neurological conditions to consider (differential diagnosis)
        3. Recommended additional tests or examinations
        4. Possible anatomical correlates of the symptoms
        
        Format your response as a clinical assessment a medical professional would use.
        """
        
        # Get interpretation
        response = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=800,
            temperature=0,
            system="You are an expert neurologist providing clinical interpretation of neurological symptoms.",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        
        return response.content[0].text
    
    except Exception as e:
        return f"Error interpreting symptoms: {str(e)}"

# Create langchain tools
brain_imaging_tool = Tool(
    name="ImagingAnalyzer",
    func=analyze_imaging,
    description="Analyzes imaging data to extract findings and interpretations"
)

symptoms_tool = Tool(
    name="NeurologicalSymptomInterpreter",
    func=interpret_neurological_symptoms,
    description="Interprets neurological symptoms to provide clinical assessment"
)

# AI Agents for Neuroscience Research  
### Automating Brain Imaging Analysis & Symptom Interpretation  

## Overview  

This notebook demonstrates how AI-powered agents can assist neuroscience researchers in automating key aspects of brain imaging analysis, symptom interpretation, literature review, and treatment recommendations. By leveraging large language models (LLMs) and structured AI workflows, this system provides a streamlined approach to clinical decision support.  

### **Agents and Responsibilities**  

1. **Neuroradiologist** – Analyzes functional and structural brain imaging (fMRI/MRI) to identify abnormalities and assess neural function.  
2. **Clinical Neurologist** – Evaluates neuroimaging results alongside patient symptoms to provide a differential diagnosis.  
3. **Medical Research Scientist** – Identifies and summarizes recent research literature relevant to the diagnosed condition and emerging treatments.  
4. **Neuropharmacologist** – Develops a treatment plan based on the neurologist’s diagnosis and the latest research findings.  

### **Workflow and Collaboration**  

The agents operate in a structured pipeline, ensuring that each stage builds upon prior results:  

1. **Neuroradiologist** – Processes brain imaging data and identifies abnormalities.  
2. **Clinical Neurologist** – Uses imaging findings and patient symptoms to generate a differential diagnosis.  
3. **Medical Research Scientist** – Conducts a literature review to provide supporting evidence for diagnosis and treatment considerations.  
4. **Neuropharmacologist** – Develops a treatment plan incorporating both clinical findings and research insights.  

This multi-agent system models real-world clinical workflows, integrating radiology, neurology, research, and pharmacology into an AI-driven support tool.  

## AI-Powered Neuroscience Agents  

This system defines four **AI-powered agents** designed to support neuroscience research and clinical decision-making:  

1. **Neuroradiologist** – Analyzes fMRI and MRI brain imaging data to identify structural and functional abnormalities.  
2. **Clinical Neurologist** – Interprets neurological symptoms and imaging findings to provide a differential diagnosis.  
3. **Medical Research Scientist** – Conducts literature reviews to identify relevant studies and emerging treatment options.  
4. **Neuropharmacologist** – Develops treatment recommendations based on clinical findings and current research.  

### **AI Agents and Their Functionality**  
Each AI agent is a **goal-driven system** that operates within a defined role, using structured data and computational models to complete specialized tasks.  

- **Defined Role** – Each agent is assigned a specific domain of expertise.  
- **Task Execution** – Agents use **predefined tools** (functions) and **large language models (LLMs)** to process neuroscience data.  
- **Decision Support** – Outputs from these agents assist researchers and clinicians in analyzing imaging data, forming diagnoses, reviewing literature, and recommending treatments.  

This structured approach allows AI agents to function as **automated assistants**, augmenting human expertise in neuroscience research and clinical workflows.  

In [ ]:
from crewai import Agent, Task, Crew
from langchain.tools import Tool

# Brain Imaging Specialist
neuroradiologist = Agent(
    role="Imaging Specialist",
    goal="Analyze fMRI scans to detect abnormalities and predict functional impairments.",
    backstory="You are an expert in neuroimaging, specializing in fMRI and structural MRI analysis. Your role is to assess brain regions and identify activity patterns.",
    verbose=False,
    llm_config={"model": "claude-3-5-sonnet-20240620"}
)

# Clinical Neurologist
clinical_neurologist = Agent(
    role="Clinical Neurologist",
    goal="Provide a differential diagnosis based on patient symptoms and neuroimaging results.",
    backstory="You specialize in neurology and diagnosing conditions based on clinical presentation and imaging.",
    verbose=False,
    llm_config={"model": "claude-3-5-sonnet-20240620"}
)

# AI Research Assistant
ai_research_assistant = Agent(
    role="AI Research Assistant",
    goal="Retrieve recent medical literature related to the neurological condition diagnosed.",
    backstory="You are an AI that searches medical research papers to provide the latest insights.",
    verbose=False,
    llm_config={"model": "claude-3-5-sonnet-20240620"}
)

# Neuropharmacologist
neuropharmacologist = Agent(
    role="Neuropharmacologist",
    goal="Recommend treatment strategies based on the neurologist's diagnosis.",
    backstory="You specialize in neuropharmacology, creating treatment plans based on diagnosed neurological disorders.",
    verbose=False,
    llm_config={"model": "claude-3-5-sonnet-20240620"}
)


## Defining AI-Driven Neuroscience Tasks  

Each AI agent is assigned a **specialized neuroscience task** using `crewai.Task`, ensuring that outputs are structured and actionable.  

### **What is a Task?**  
A `Task` defines a **specific objective** that an AI agent must complete, structured with:  
- **A detailed description** that outlines the scope and required input data.  
- **An assigned agent** responsible for executing the task.  
- **An expected output** to ensure the task delivers relevant and interpretable results.  

### **Neuroscience Tasks and Assignments:**  
Each task corresponds to a real-world research or clinical function:  

1. **Brain Imaging Analysis** – The neuroradiologist evaluates fMRI/MRI scans to identify structural and functional abnormalities.  
2. **Neurological Symptom Assessment** – The clinical neurologist interprets patient symptoms and imaging findings to form a differential diagnosis.  
3. **Literature Review** – The medical research scientist identifies recent studies relevant to the diagnosed condition and emerging treatments.  
4. **Treatment Recommendation** – The neuropharmacologist formulates a treatment plan based on clinical findings and supporting research.  

By structuring these tasks within a multi-agent system, this framework simulates a **collaborative neuroscience workflow**, integrating radiology, clinical neurology, medical research, and pharmacology into an AI-driven decision-support tool.  

In [ ]:
# Get JSON strings for data
brain_scan_json = json.dumps(brain_scan_data, ensure_ascii=False, indent=2)
symptoms_json = json.dumps(patient_symptoms, ensure_ascii=False, indent=2)

# Task: Analyze fMRI scans
imaging_task = Task(
    description=f"Analyze the following fMRI scans to detect abnormalities and predict functional impairments:\n\n{brain_scan_json}",
    agent=neuroradiologist,
    expected_output="Detailed fMRI analysis report highlighting functional impairments."
)

# Task: Provide differential diagnosis
neurological_diagnosis_task = Task(
    description=f"""
    Given the following fMRI scan results and patient symptoms, provide a differential diagnosis:

    fMRI Scan Data:
    {brain_scan_json}

    Patient Symptoms:
    {symptoms_json}
    """,
    agent=clinical_neurologist,
    expected_output="Neurological diagnosis report including possible conditions and their severity."
)

# Task: Retrieve recent medical research
research_task = Task(
    description="""
    Search for and summarize the latest medical research related to the diagnosed neurological condition.

    Neurological Diagnosis:
    {neurological_diagnosis_task.expected_output}

    Goal:
    - Identify **3-5 recent medical papers** related to the diagnosis.
    - Provide **brief, structured summaries** of the findings.
    - Focus on **treatment options, new therapies, and clinical trials**.
    """,
    agent=ai_research_assistant,
    expected_output="Summarized research on the latest treatment strategies for the diagnosed condition."
)

# Task: Recommend treatment options
treatment_recommendation_task = Task(
    description="""
    Given the following fMRI scan findings, neurological diagnosis, and latest research, recommend a treatment strategy.

    Brain Imaging Analysis Results:
    {brain_imaging_task.expected_output}

    Neurological Diagnosis:
    {neurological_diagnosis_task.expected_output}

    Recent Medical Research Summaries:
    {research_task.expected_output}

    Goal:
    - Provide a structured treatment plan, including medication and therapy recommendations.
    - If applicable, suggest **non-pharmaceutical interventions** such as lifestyle changes or rehabilitation.
    - Consider recent medical research when making recommendations.
    """,
    agent=neuropharmacologist,
    expected_output="A structured treatment plan including medication, therapy, and research-backed recommendations."
)

## Coordinating AI Agents with CrewAI  

We now execute both AI agents **simultaneously** using `CrewAI`.  

### **What is a Crew?**  
A `Crew` is a multi-agent system where each agent:  
- **Receives a task** based on its expertise.  
- **Processes input data** and generates an AI-driven analysis.  
- **Works independently** but contributes to the overall workflow.  

By running the `kickoff()` method, we launch the AI agents and obtain structured results.

In [ ]:
# Create the neuroscience crew
neuroscience_crew = Crew(
    agents=[neuroradiologist, clinical_neurologist, ai_research_assistant, neuropharmacologist],
    tasks=[imaging_task, neurological_diagnosis_task, research_task, treatment_recommendation_task],
    verbose=False  # Quiet execution
)

# Run the multi-agent analysis
print("Starting advanced neuroscience analysis...\n")
result = neuroscience_crew.kickoff()

if not result:
    print("ERROR")
else:
    print("\n\nAnalysis Complete...")
# print(result)

In [ ]:
# Extract structured results from CrewOutput
task_outputs = result.tasks_output

# Initialize formatted results
formatted_results = """
══════════════════════════════════════════
  ADVANCED NEUROSCIENCE ANALYSIS  
══════════════════════════════════════════
"""

# Loop through task outputs and extract results
for task in task_outputs:
    agent_name = task.agent
    task_result = task.raw if task.raw else "No result available."

    formatted_results += f"""
══════════════════════════════════════════
{agent_name} Analysis  
══════════════════════════════════════════

{task_result}

"""

# Save results as a structured text file
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"report_{ts}.txt"
with open(output_file, "w", encoding="utf-8") as file:
    file.write(formatted_results)

print(f"\nResults saved to: {output_file}")


## Visualize Agent Workflow

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create a simple diagram showing the agent workflow
def visualize_agent_workflow():
    G = nx.DiGraph()
    
    # Add nodes for data sources
    G.add_node("Brain Imaging Data", type="data")
    G.add_node("Neurological Symptoms", type="data")
    
    # Add nodes for agents
    G.add_node("Brain Imaging Analyst", type="agent")
    G.add_node("Clinical Neurologist", type="agent")
    
    # Add nodes for outputs
    G.add_node("Imaging Analysis", type="output")
    G.add_node("Clinical Assessment", type="output")
    
    # Add edges
    G.add_edge("Brain Imaging Data", "Brain Imaging Analyst")
    G.add_edge("Neurological Symptoms", "Clinical Neurologist")
    G.add_edge("Brain Imaging Analyst", "Imaging Analysis")
    G.add_edge("Clinical Neurologist", "Clinical Assessment")
    
    # Set up the plot
    plt.figure(figsize=(10, 6))
    pos = nx.spring_layout(G, seed=42)
    
    # Draw nodes with different colors based on type
    data_nodes = [n for n, d in G.nodes(data=True) if d.get('type') == 'data']
    agent_nodes = [n for n, d in G.nodes(data=True) if d.get('type') == 'agent']
    output_nodes = [n for n, d in G.nodes(data=True) if d.get('type') == 'output']
    
    nx.draw_networkx_nodes(G, pos, nodelist=data_nodes, node_color='skyblue', node_size=2000, alpha=0.8)
    nx.draw_networkx_nodes(G, pos, nodelist=agent_nodes, node_color='lightgreen', node_size=2000, alpha=0.8)
    nx.draw_networkx_nodes(G, pos, nodelist=output_nodes, node_color='salmon', node_size=2000, alpha=0.8)
    
    # Draw edges and labels
    nx.draw_networkx_edges(G, pos, width=1.5, alpha=0.7, arrows=True, arrowsize=20)
    nx.draw_networkx_labels(G, pos, font_size=11, font_weight='bold')
    
    # Add a legend
    plt.plot([0], [0], 'o', markersize=15, color='skyblue', alpha=0.8, label='Input Data')
    plt.plot([0], [0], 'o', markersize=15, color='lightgreen', alpha=0.8, label='Agent')
    plt.plot([0], [0], 'o', markersize=15, color='salmon', alpha=0.8, label='Output')
    plt.legend()
    
    plt.title('Neuroscience Agent Workflow', size=15)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize the workflow
visualize_agent_workflow()

## Future Applications

This simple demonstration shows how AI agents can assist with routine neuroscience tasks. More advanced applications could include:

1. **Automated Literature Analysis** - Agents that scan research papers to extract findings
2. **Experimental Design Assistant** - Agents that help design robust neuroscience experiments
3. **Data Processing Pipeline** - Agents that preprocess neuroimaging data following best practices
4. **Research Collaboration** - Multi-agent systems that facilitate collaboration between specialists

As AI capabilities continue to advance, these systems could significantly accelerate neuroscience research while maintaining scientific rigor.